In [19]:
import pandas as pd
import numpy as np
import yfinance as yf
from sqlalchemy import create_engine, Table, MetaData
from datetime import datetime
from warnings import filterwarnings
filterwarnings('ignore')

def select_database(database_name):

    db = database_name

    return db

def create_db_engine(user, password, host_name, database_name):
    """
    Creates a connection to the MySQL database.
    """
    init_engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host_name}/{database_name}')
    return init_engine

def connect_to_engine(engine):
    conn = engine.connect()
    return conn

In [2]:
user_name = "postgres"
user_password = "root"
host = "localhost"
port = "5432"
database_name = "client_db_test"

# PostgreSQL connection string
postgres_connection_string = f"postgresql://{user_name}:{user_password}@{host}:{port}/{database_name}"

# Create the connection to the database
engine = create_engine(postgres_connection_string)

conn = connect_to_engine(engine)

In [3]:
metadata = MetaData()
metadata.reflect(bind=engine)

# __Initial DB Functions__

In [22]:
def initial_fetch_data(symbol, date, conn):

    if symbol == "^NSEI" or symbol == "^NSEBANK":
        symbol = symbol.upper()
    else:
        symbol = f"{symbol}.NS".upper()

    date = datetime.strptime(date, '%Y-%m-%d')
    # temp_end = datetime.strptime("2024-01-31", '%Y-%m-%d')

    # Download 5-minute data
    min5 = yf.download(symbol, start=date, interval='5m')
    min5.index = min5.index.strftime('%Y-%m-%d %H:%M:%S') 
    min5.index = pd.to_datetime(min5.index)
    min5['Date'] = min5.index.date
    for i in range(len(min5.columns)):
        min5.columns.values[i] = "5m_" + min5.columns.values[i]
    min5.rename(columns={'5m_Date':'Date'}, inplace=True)
    min5 = min5.reindex(columns=['Date','5m_Open', '5m_High', '5m_Low', '5m_Close', '5m_Adj Close', '5m_Volume'])

    # Download 15-minute data
    min15 = yf.download(symbol, start=date, interval='15m')
    min15.index = min15.index.strftime('%Y-%m-%d %H:%M:%S')
    min15.index = pd.to_datetime(min15.index)
    min15['Date'] = min15.index.date
    for i in range(len(min15.columns)):
        min15.columns.values[i] = "15m_" + min15.columns.values[i]
    min15.rename(columns={'15m_Date':'Date'}, inplace=True)
    min15 = min15.reindex(columns=['Date','15m_Open', '15m_High', '15m_Low', '15m_Close', '15m_Adj Close', '15m_Volume'])
    min15 = min15.resample('5T').ffill()

    # Download 60-minute data
    min60 = yf.download(symbol, start=date, interval='60m')
    min60.index = min60.index.strftime('%Y-%m-%d %H:%M:%S')
    min60.index = pd.to_datetime(min60.index)
    min60['Date'] = min60.index.date
    for i in range(len(min60.columns)):
        min60.columns.values[i] = "60m_" + min60.columns.values[i]
    min60.rename(columns={'60m_Date':'Date'}, inplace=True)
    min60 = min60.reindex(columns=['Date','60m_Open', '60m_High', '60m_Low', '60m_Close', '60m_Adj Close', '60m_Volume'])
    min60 = min60.resample('5T').ffill()

    # Download 60-minute data
    day1 = yf.download(symbol, start=date, interval='1d')
    day1.index = day1.index.strftime('%Y-%m-%d %H:%M:%S')
    day1.index = pd.to_datetime(day1.index)
    day1['Date'] = day1.index.date
    for i in range(len(day1.columns)):
        day1.columns.values[i] = "1d_" + day1.columns.values[i]
    day1.rename(columns={'1d_Date':'Date'}, inplace=True)
    day1 = day1.reindex(columns=['Date','1d_Open', '1d_High', '1d_Low', '1d_Close', '1d_Adj Close', '1d_Volume'])
    day1 = day1.resample('5T').ffill()

    # Concatenate dataframes
    main_df = pd.concat([min5, min15, min60, day1], axis=1)
    # Drop duplicate columns
    main_df = main_df.loc[:, ~main_df.columns.duplicated()]
    # pd.set_option('display.max_rows', None)
    main_df = main_df.dropna(subset=['5m_Open','5m_High','5m_Low','5m_Close'])
    
    if symbol == "^NSEI" or symbol == "^NSEBANK":
        main_df.to_sql(symbol, conn, if_exists='append', index=True, index_label='Datetime')
    else:
        main_df.to_sql(symbol[:-3], conn, if_exists='append', index=True, index_label='Datetime')

    return main_df

In [23]:
initial_fetch_data("Reliance", "2024-04-01", None)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


AttributeError: 'NoneType' object has no attribute 'cursor'

# __Rerun DB Functions__

In [6]:
user_password = "root"
host = "localhost"
port = "5432"
database_name = "harish_stocks"

# PostgreSQL connection string
postgres_connection_string = f"postgresql://{user_name}:{user_password}@{host}:{port}/{database_name}"

# Create the connection to the database
engine = create_engine(postgres_connection_string)

conn = connect_to_engine(engine)

# Reflect the database tables
metadata = MetaData()
metadata.reflect(bind=engine)

table_names_list = metadata.tables.keys()


In [7]:
print(table_names_list)

print(len(table_names_list))

dict_keys(['ipcalab', 'jswsteel', 'jublfood', 'bpcl', 'motherson', 'jswenergy', 'britannia', 'tataelxsi', 'tatamotors', 'tatamtrdvr', 'itc', 'jindalstel', 'ramcocem', 'recltd', 'infy', 'epigral', 'hindalco', 'hindpetro', 'axisbank', 'marico', 'maruti', 'asianpaint', 'lodha', 'lt', 'astral', 'aubank', 'auropharma', 'torntpharm', 'escorts', 'fact', 'fluorochem', 'gail', 'boschltd', 'abb', 'industower', 'ntpc', 'nykaa', 'gujgasltd', 'hal', 'havells', 'bajaj-auto', 'bajajfinsv', 'bajfinance', 'ioc', 'tatapower', 'godrejprop', 'dabur', 'piind', 'tcs', 'techm', 'tiindia', 'navinfluor', 'nestleind', 'nhpc', 'bhel', 'sail', 'dalbharat', 'deepakntr', 'msumi', 'kotakbank', 'ambujacem', 'acc', 'acl', 'adanient', 'adaniports', 'upl', 'vbl', 'vedl', 'coromandel', 'm&mfin', 'mankind', 'apollohosp', 'apollotyre', 'pidilitind', 'titan', 'gland', 'kpittech', 'l&tfh', 'latentview', 'lauruslabs', 'lichsgfin', 'lici', 'abfrl', 'eichermot', 'hindunilvr', 'icicibank', 'icicigi', 'icicipruli', 'idfcfirstb', 

In [8]:
# def fetch_table_names(engine, metadata):
#     """
#     Fetches the table names from the database.
#     """
#     metadata.reflect(engine)

#     table_names = metadata.tables.keys()

#     table_list = []
#     for table_name in table_names:
#         table_list.append(table_name)

#     return table_list

In [9]:
# def fetch_db_data(conn, table_name):
#     """
#     Fetches data from the database.
#     """
#     data = pd.read_sql(f"SELECT * FROM `{table_name.upper()}`", conn)
#     return data

In [10]:
# def rerun_fetch_data(symbol, date):

#     if symbol == "^NSEI" or symbol == "^NSEBANK":
#         symbol = symbol.upper()
#     else:
#         symbol = f"{symbol}.NS".upper()
        
#     date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')

#     # Download 5-minute data
#     min5 = yf.download(symbol, start=date, interval='5m')
#     min5.index = min5.index.strftime('%Y-%m-%d %H:%M:%S') 
#     min5.index = pd.to_datetime(min5.index)
#     min5['Date'] = min5.index.date
#     for i in range(len(min5.columns)):
#         min5.columns.values[i] = "5m_" + min5.columns.values[i]
#     min5.rename(columns={'5m_Date':'Date'}, inplace=True)
#     min5 = min5.reindex(columns=['Date','5m_Open', '5m_High', '5m_Low', '5m_Close', '5m_Adj Close', '5m_Volume'])

#     # Download 15-minute data
#     min15 = yf.download(symbol, start=date, interval='15m')
#     min15.index = min15.index.strftime('%Y-%m-%d %H:%M:%S')
#     min15.index = pd.to_datetime(min15.index)
#     min15['Date'] = min15.index.date
#     for i in range(len(min15.columns)):
#         min15.columns.values[i] = "15m_" + min15.columns.values[i]
#     min15.rename(columns={'15m_Date':'Date'}, inplace=True)
#     min15 = min15.reindex(columns=['Date','15m_Open', '15m_High', '15m_Low', '15m_Close', '15m_Adj Close', '15m_Volume'])
#     min15 = min15.resample('5T').ffill()

#     # Download 15-minute data
#     min60 = yf.download(symbol, start=date, interval='60m')
#     min60.index = min60.index.strftime('%Y-%m-%d %H:%M:%S')
#     min60.index = pd.to_datetime(min60.index)
#     min60['Date'] = min60.index.date
#     for i in range(len(min60.columns)):
#         min60.columns.values[i] = "60m_" + min60.columns.values[i]
#     min60.rename(columns={'60m_Date':'Date'}, inplace=True)
#     min60 = min60.reindex(columns=['Date','60m_Open', '60m_High', '60m_Low', '60m_Close', '60m_Adj Close', '60m_Volume'])
#     min60 = min60.resample('5T').ffill()

#         # Download 60-minute data
#     day1 = yf.download(symbol, start=date, interval='1d')
#     day1.index = day1.index.strftime('%Y-%m-%d %H:%M:%S')
#     day1.index = pd.to_datetime(day1.index)
#     day1['Date'] = day1.index.date
#     for i in range(len(day1.columns)):
#         day1.columns.values[i] = "1d_" + day1.columns.values[i]
#     day1.rename(columns={'1d_Date':'Date'}, inplace=True)
#     day1 = day1.reindex(columns=['Date','1d_Open', '1d_High', '1d_Low', '1d_Close', '1d_Adj Close', '1d_Volume'])
#     day1 = day1.resample('5T').ffill()

#     # Concatenate dataframes
#     main_df = pd.concat([min5, min15, min60, day1], axis=1)
#     # Drop duplicate columns
#     main_df = main_df.loc[:, ~main_df.columns.duplicated()]
#     # pd.set_option('display.max_rows', None)
#     main_df = main_df.dropna(subset=['5m_Open','5m_High','5m_Low','5m_Close'])
    
#     return main_df

In [16]:
def rerun_fetch_data(symbol, date):

    if symbol == "^NSEI" or symbol == "^NSEBANK":
        symbol = symbol.upper()
    else:
        symbol = f"{symbol}.NS".upper()
        
    date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')

    # Download 5-minute data
    min5 = yf.download(symbol, start=date, interval='5m')
    min5.index = min5.index.strftime('%Y-%m-%d %H:%M:%S') 
    min5.index = pd.to_datetime(min5.index)
    min5['Date'] = min5.index.date
    for i in range(len(min5.columns)):
        min5.columns.values[i] = "5m_" + min5.columns.values[i]
    min5.rename(columns={'5m_Date':'Date'}, inplace=True)
    min5 = min5.reindex(columns=['Date','5m_Open', '5m_High', '5m_Low', '5m_Close', '5m_Adj Close', '5m_Volume'])

    # Download 15-minute data
    min15 = yf.download(symbol, start=date, interval='15m')
    min15.index = min15.index.strftime('%Y-%m-%d %H:%M:%S')
    min15.index = pd.to_datetime(min15.index)
    min15['Date'] = min15.index.date
    for i in range(len(min15.columns)):
        min15.columns.values[i] = "15m_" + min15.columns.values[i]
    min15.rename(columns={'15m_Date':'Date'}, inplace=True)
    min15 = min15.reindex(columns=['Date','15m_Open', '15m_High', '15m_Low', '15m_Close', '15m_Adj Close', '15m_Volume'])
    min15 = min15.resample('5T').ffill()

    # Download 60-minute data
    min60 = yf.download(symbol, start=date, interval='60m')
    min60.index = min60.index.strftime('%Y-%m-%d %H:%M:%S')
    min60.index = pd.to_datetime(min60.index)
    min60['Date'] = min60.index.date
    for i in range(len(min60.columns)):
        min60.columns.values[i] = "60m_" + min60.columns.values[i]
    min60.rename(columns={'60m_Date':'Date'}, inplace=True)
    min60 = min60.reindex(columns=['Date','60m_Open', '60m_High', '60m_Low', '60m_Close', '60m_Adj Close', '60m_Volume'])
    min60 = min60.resample('5T').ffill()




    # Download 1-day data
    day1 = yf.download(symbol, start=date, interval='1d')

    # print(f"\n Raw data for {symbol} is {day1}\n\n")
    day1.index = day1.index.strftime('%Y-%m-%d %H:%M:%S')
    day1.index = pd.to_datetime(day1.index)
    day1['Date'] = day1.index.date
    for i in range(len(day1.columns)):
        day1.columns.values[i] = "1d_" + day1.columns.values[i]

    # print(f"\n Raw data after created date column for {symbol} is {day1}\n\n")

    day1.rename(columns={'1d_Date':'Date'}, inplace=True)
    day1 = day1.reindex(columns=['Date','1d_Open', '1d_High', '1d_Low', '1d_Close', '1d_Adj Close', '1d_Volume'])



    # Extend the index to the end of the day
    last_timestamp = day1.index[-1]
    end_of_day = last_timestamp.replace(hour=23, minute=59, second=59)
    new_index = pd.date_range(start=day1.index[0], end=end_of_day, freq='T')
    day1 = day1.reindex(new_index)



    # print(f"\n final data after reindexing columns before splitting it into 5 mins for {symbol} is {day1}\n\n")
    day1 = day1.resample('5min').ffill()
    day1.fillna(method='ffill', inplace=True)

    # print(f"\n final data after resampling for {symbol} is {day1}\n\n")




    # Concatenate dataframes
    main_df = pd.concat([min5, min15, min60, day1], axis=1)
    # Drop duplicate columns
    main_df = main_df.loc[:, ~main_df.columns.duplicated()]
    # pd.set_option('display.max_rows', None)
    main_df = main_df.dropna(subset=['5m_Open','5m_High','5m_Low','5m_Close'])

    # print(f"\n Full & final data for {symbol} is {main_df}\n\n")
    
    return main_df


In [12]:
table_name = "reliance"

In [17]:
df = pd.read_sql_table(table_name, con=engine)

# Find the starting index of the latest chunk of consecutive null-filled values
null_chunks = df['60m_Open'].isnull().astype(int).diff().fillna(0)
latest_chunk_start_index = null_chunks[null_chunks == 1].index[-1]

# Getting time for it
date_value = df.loc[latest_chunk_start_index, 'Datetime']
print(date_value)
print(type(date_value))

# split data according to the date in two parts
proper_df = df[df['Datetime'] < date_value]
null_values_df = df[df['Datetime'] >= date_value]

datetime_value = df.loc[proper_df.index[-1], 'Datetime']

after_df = rerun_fetch_data(table_name, datetime_value)
after_df.reset_index(inplace=True)

final_df = pd.concat([proper_df, after_df])
final_df.reset_index(drop=True, inplace=True)

[*********************100%%**********************]  1 of 1 completed

2024-05-22 15:20:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [18]:
final_df.head(50)

,Datetime,Date,5m_Open,5m_High,5m_Low,5m_Close,5m_Adj Close,5m_Volume,15m_Open,15m_High,...,60m_Close,60m_Adj Close,60m_Volume,index,1d_Open,1d_High,1d_Low,1d_Close,1d_Adj Close,1d_Volume
0,2024-01-01 09:15:00,2024-01-01 00:00:00,2582.000000,2589.699951,2581.000000,2581.000000,2581.000000,0.0,2582.000000,2589.699951,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-01 09:20:00,2024-01-01 00:00:00,2580.550049,2583.000000,2577.000000,2579.199951,2579.199951,57683.0,2582.000000,2589.699951,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-01 09:25:00,2024-01-01 00:00:00,2579.050049,2579.449951,2574.000000,2576.000000,2576.000000,48091.0,2582.000000,2589.699951,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-01 09:30:00,2024-01-01 00:00:00,2576.000000,2580.800049,2575.000000,2580.000000,2580.000000,31590.0,2576.000000,2584.000000,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-01 09:35:00,2024-01-01 00:00:00,2580.000000,2581.850098,2575.500000,2575.500000,2575.500000,32074.0,2576.000000,2584.000000,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-01-01 09:40:00,2024-01-01 00:00:00,2576.649902,2584.000000,2575.100098,2582.050049,2582.050049,29238.0,2576.000000,2584.000000,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-01-01 09:45:00,2024-01-01 00:00:00,2582.250000,2587.500000,2582.000000,2586.500000,2586.500000,24670.0,2582.250000,2587.500000,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
7,2024-01-01 09:50:00,2024-01-01 00:00:00,2586.500000,2587.250000,2584.949951,2586.199951,2586.199951,31853.0,2582.250000,2587.500000,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
8,2024-01-01 09:55:00,2024-01-01 00:00:00,2585.100098,2586.000000,2583.600098,2584.100098,2584.100098,21865.0,2582.250000,2587.500000,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-01-01 10:00:00,2024-01-01 00:00:00,2584.100098,2586.250000,2582.899902,2583.300049,2583.300049,26170.0,2584.100098,2586.250000,...,2581.600098,2581.600098,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN
